# Qcodes example with Keithley 2600

In [1]:
%matplotlib nbagg
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc

qc.halt_bg()
qc.set_mp_method('spawn')  # force Windows behavior on mac

# this makes a widget in the corner of the window to show and control
# subprocesses and any output they would print to the terminal
qc.show_subprocess_widget()

<IPython.core.display.Javascript object>

No loop running


In [2]:
import qcodes.instrument_drivers.tektronix.Keithley_2600 as keith

In [3]:
# spawn doesn't like function or class definitions in the interpreter
# session - had to move them to a file.

# now create this "experiment"
k1 = keith.Keithley_2600('Keithley1', 'GPIB0::15::INSTR',channel='a')#,server_name=None)
k2 = keith.Keithley_2600('Keithley2', 'GPIB0::15::INSTR',channel='b')#,server_name=None)

station = qc.Station(k1,k2)

# could measure any number of things by adding arguments to this
# function call, but here we're just measuring one, the meter amplitude
station.set_measurement(k1.curr,k2.curr)

# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice
# this could be simplified to a station method that gathers all parameters
# and adds them all as (disambiguated) globals, printing what it did
# something like:
#   station.gather_parameters(globals())

vsd1, vsd2, curr1, curr2 = k1.volt, k2.volt, k1.curr, k2.curr

# once we have implemented a monitor, defining a station will start a
# DataServer process, and you would see it in the subprocess widget,
# or via active_children() as here:
# qc.active_children()

In [4]:
station.snapshot()

{'instruments': {'Keithley1': {'functions': {},
   'metadata': {'info': {'model': '2614B',
     'serial_number': '4083825',
     'software_revision': '3.2.1',
     'vendor': 'Keithley Instruments Inc.'}},
   'parameters': {'curr': {'ts': None, 'value': None},
    'limiti': {'ts': None, 'value': None},
    'limitv': {'ts': None, 'value': None},
    'mode': {'ts': None, 'value': None},
    'output': {'ts': None, 'value': None},
    'rangei': {'ts': None, 'value': None},
    'rangev': {'ts': None, 'value': None},
    'volt': {'ts': None, 'value': None}}},
  'Keithley2': {'functions': {},
   'metadata': {'info': {'model': '2614B',
     'serial_number': '4083825',
     'software_revision': '3.2.1',
     'vendor': 'Keithley Instruments Inc.'}},
   'parameters': {'curr': {'ts': None, 'value': None},
    'limiti': {'ts': None, 'value': None},
    'limitv': {'ts': None, 'value': None},
    'mode': {'ts': None, 'value': None},
    'output': {'ts': None, 'value': None},
    'rangei': {'ts': None,

In [5]:
# we can get the measured quantities right now
station.measure()

[2.98023e-13, -9.41753e-13]

In [6]:
# start a Loop (which by default runs in a seprarate process)
# the sweep values are defined by slicing the parameter object
# but more complicated sweeps (eg nonlinear, or adaptive) can
# easily be used instead

# Notice that I'm using an explicit location and `overwrite=True` here so that
# running this notebook over and over won't result in extra files.
# But if you leave these out, you get a new timestamped DataSet each time.
data = qc.Loop(vsd1[-5:5:0.5], 0.03).run(location='testsweep', overwrite=True)

# now there should be two extra processes running, DataServer and a sweep
# I'll omit the active_children call now because you can see them in the
# subprocess widget

DataSet: DataMode.PULL_FROM_SERVER, location='testsweep'
   curr_0: curr
   curr_1: curr
   volt: volt
started at 2016-04-20 15:37:23


In [7]:
# manually bring the data into the main process and display it as numbers
data.sync()
data.arrays

{'curr_0': DataArray[20]: curr_0
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]),
 'curr_1': DataArray[20]: curr_1
 array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]),
 'volt': DataArray[20]: volt
 array([ -5.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
         nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan])}

In [8]:
# live-updating plot, that syncs the data and stops updating when it's finished
# plot = qc.MatPlot(data.amplitude)
plotQ1 = qc.QtPlot(data.volt, data.curr_0)
plotQ2 = qc.QtPlot(data.volt, data.curr_1)

In [9]:
data2 = qc.Loop(vsd1[-5:5:0.1],0).each(
                qc.Loop(vsd2[-2:2:.2], 0)
).run(location='test2d', overwrite=True)

# use the subplot and add features of qc.MatPlot
# plot2 = qc.MatPlot(data2.amplitude_0, cmap=plt.cm.hot, figsize=(12, 4.5), subplots=(1, 2))
# plot2.add(data2.amplitude_3, cmap=plt.cm.hot, subplot=2)

# the equivalent in QtPlot
plot2Q = qc.QtPlot(data2.curr_1, figsize=(1200, 500))
# plot2Q.add(data2.curr_1, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test2d'
   curr_1: curr
   volt: volt
   curr_0: curr
   volt_0: volt
started at 2016-04-20 15:22:46


In [19]:
k1.setattr(('metadata','test'), 11)
k1.getattr('info')

{'model': '2614B',
 'serial_number': '4083825',
 'software_revision': '3.2.1',
 'vendor': 'Keithley Instruments Inc.'}

In [8]:
k1.ask_direct('*IDN?')

'Keithley Instruments Inc., Model 2614B, 4083825, 3.2.1'

In [18]:
station.snapshot(update=True)

{'instruments': {'Keithley1': {'functions': {},
   'metadata': {'test': 11},
   'parameters': {'curr': {'ts': '2016-04-20 15:31:51', 'value': -2.14577e-13},
    'limiti': {'ts': '2016-04-20 15:31:51', 'value': 0.1},
    'limitv': {'ts': '2016-04-20 15:31:51', 'value': 20.0},
    'mode': {'ts': '2016-04-20 15:31:51', 'value': 'voltage'},
    'output': {'ts': '2016-04-20 15:31:51', 'value': 'ON'},
    'rangei': {'ts': '2016-04-20 15:31:51', 'value': 1e-07},
    'rangev': {'ts': '2016-04-20 15:31:51', 'value': 20.0},
    'volt': {'ts': '2016-04-20 15:31:51', 'value': 4.90016}}},
  'Keithley2': {'functions': {},
   'parameters': {'curr': {'ts': '2016-04-20 15:31:51', 'value': -1.07288e-12},
    'limiti': {'ts': '2016-04-20 15:31:51', 'value': 0.1},
    'limitv': {'ts': '2016-04-20 15:31:51', 'value': 20.0},
    'mode': {'ts': '2016-04-20 15:31:51', 'value': 'voltage'},
    'output': {'ts': '2016-04-20 15:31:51', 'value': 'ON'},
    'rangei': {'ts': '2016-04-20 15:31:51', 'value': 1e-07},
 

In [12]:
plot2Q.add(data2.curr_0, subplot=2)


In [18]:
data3 = qc.Loop(vsd1[-15:15:1], 0).each(
                qc.Loop(vsd2[-5:5:1], 0),
                qc.Loop(vsd2[5:-5:1], 0),
).run(location='test_multi_d', overwrite=True)

# several plots updating simultaneously
# plot3 = qc.MatPlot(data3.curr_1_1, cmap=plt.cm.hot)
# plot3b = qc.MatPlot(data3.curr_1_0, cmap=plt.cm.hot, figsize=(12, 4.5), subplots=(1,2))
# plot3b.add(data3.curr_0_1, subplot=2)
plot3Q = qc.QtPlot(data3.curr_1)
plot3bQ = qc.QtPlot(data3.curr_1, figsize=(1200, 500))
plot3bQ.add(data3.curr_0, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test_multi_d'
   curr_0_1: curr
   volt_0: volt
   curr_0_0: curr
   volt_1: volt
   curr_1_0: curr
   curr_1_1: curr
   volt: volt
started at 2016-04-20 14:09:16


AttributeError: 'DataSet' object and its delegates have no attribute 'curr_1'

In [9]:
# An example of a parameter that returns several values of different dimension
# This produces the last two arrays from data3, but only takes the data once.
data4 = qc.Loop(c1[-15:15:1], 0.1).each(
    AverageAndRaw(meter.amplitude, c2[-10:10:0.2], 0.001)
).run(location='test_complex_param', overwrite=True)

# plot4 = qc.MatPlot(data4.amplitude, cmap=plt.cm.hot, subplots=(1,2), figsize=(12, 4.5))
# plot4.add(data4.avg_amplitude, subplot=2)
plot4Q = qc.QtPlot(data4.amplitude, figsize=(1200, 500))
plot4Q.add(data4.avg_amplitude, subplot=2)

DataSet: DataMode.PULL_FROM_SERVER, location='test_complex_param'
   avg_amplitude: avg_amplitude
   chan2: chan2
   chan1: chan1
   amplitude: amplitude
started at 2016-02-02 12:18:09
